In [1]:
import pandas as pd
import IP2Location
import os
import re


In [2]:
def ips_unicos(df: pd.DataFrame) -> pd.DataFrame:
    ips = []
    
    for c in df.columns:
        if 'ip' in c.lower():
            ips.extend(df[c].unique())
            
    return ips

def pais(ip: str) -> str:
    return region_db.get_region_code(database.get_all(ip).country_short, database.get_all(ip).region)

def conjunto(ips: list) -> pd.DataFrame:
    countries = []
    
    for i in ips:
        countries.append(pais(i))

    return pd.DataFrame({'ip': ips, 'country': countries})

def sacar_pais(df: pd.DataFrame, ip_df: pd.DataFrame) -> pd.DataFrame:
    for c in df.columns:
        if 'ip' in c.lower():
            df = df.merge(ip_df, left_on=c, right_on='ip', how='left')
            df.drop(columns='ip', inplace=True)
            df.rename(columns={'country': c + '_country'}, inplace=True)
            
    return df

In [3]:
def leer_apache(log_file: str) -> pd.DataFrame:
    pattern = r'(?P<client_ip>\S+) (?P<ident>\S+) (?P<auth>\S+) \[(?P<timestamp>[^\]]+)\] "(?P<http_method>\S+) (?P<request_path>\S+) HTTP/(?P<http_version>\S+)" (?P<http_status>\S+) (?P<response_size>\S+) "(?P<referrer>[^\"]+)" "(?P<user_agent>[^\"]+)"'
    
    regex = re.compile(pattern)
    logs = []
    with open(log_file, 'r') as file:
        for line in file:
            match = regex.match(line.strip())
            if match:
                logs.append(match.groupdict())
    
    return pd.DataFrame(logs)

In [4]:
duv = pd.read_csv('Datos/Originales/Dataset-Unicauca-Version2-87Atts.csv', nrows=3000)
apache = leer_apache('Datos/Originales/apache-logs.txt')
snmp=pd.read_csv('Datos/Originales/reto8-registros_snmp.csv')

In [5]:
snmp.rename(columns={'SNMPv2-SMI::enterprises.9.9.599.1.3.1.1.10.0':'SNMP_IP'}, inplace=True)

In [6]:
apache['client_ip'] = apache['client_ip'].apply(lambda x: x.replace('"',""))

In [7]:
database = IP2Location.IP2Location('Datos/Originales/IP2LOCATION-LITE-DB3.BIN')
region_db = IP2Location.Region('Datos/Originales/IP2LOCATION-ISO3166-2.CSV')


In [8]:
ips = ips_unicos(apache)
ips.extend(ips_unicos(duv))
ips.extend(ips_unicos(snmp))

In [9]:
ips=list(set(ips))

In [10]:
len(ips)

24361

In [11]:
countries=conjunto(ips)

[{'code': 'US-AL', 'name': 'Alabama'}, {'code': 'US-AK', 'name': 'Alaska'}, {'code': 'US-AZ', 'name': 'Arizona'}, {'code': 'US-AR', 'name': 'Arkansas'}, {'code': 'US-CA', 'name': 'California'}, {'code': 'US-CO', 'name': 'Colorado'}, {'code': 'US-CT', 'name': 'Connecticut'}, {'code': 'US-DE', 'name': 'Delaware'}, {'code': 'US-DC', 'name': 'District of Columbia'}, {'code': 'US-FL', 'name': 'Florida'}, {'code': 'US-GA', 'name': 'Georgia'}, {'code': 'US-HI', 'name': 'Hawaii'}, {'code': 'US-ID', 'name': 'Idaho'}, {'code': 'US-IL', 'name': 'Illinois'}, {'code': 'US-IN', 'name': 'Indiana'}, {'code': 'US-IA', 'name': 'Iowa'}, {'code': 'US-KS', 'name': 'Kansas'}, {'code': 'US-KY', 'name': 'Kentucky'}, {'code': 'US-LA', 'name': 'Louisiana'}, {'code': 'US-ME', 'name': 'Maine'}, {'code': 'US-MD', 'name': 'Maryland'}, {'code': 'US-MA', 'name': 'Massachusetts'}, {'code': 'US-MI', 'name': 'Michigan'}, {'code': 'US-MN', 'name': 'Minnesota'}, {'code': 'US-MS', 'name': 'Mississippi'}, {'code': 'US-MO', 

[{'code': 'DE-BW', 'name': 'Baden-Wurttemberg'}, {'code': 'DE-BY', 'name': 'Bayern'}, {'code': 'DE-BE', 'name': 'Berlin'}, {'code': 'DE-BB', 'name': 'Brandenburg'}, {'code': 'DE-HB', 'name': 'Bremen'}, {'code': 'DE-HH', 'name': 'Hamburg'}, {'code': 'DE-HE', 'name': 'Hessen'}, {'code': 'DE-MV', 'name': 'Mecklenburg-Vorpommern'}, {'code': 'DE-NI', 'name': 'Niedersachsen'}, {'code': 'DE-NW', 'name': 'Nordrhein-Westfalen'}, {'code': 'DE-RP', 'name': 'Rheinland-Pfalz'}, {'code': 'DE-SL', 'name': 'Saarland'}, {'code': 'DE-SN', 'name': 'Sachsen'}, {'code': 'DE-ST', 'name': 'Sachsen-Anhalt'}, {'code': 'DE-SH', 'name': 'Schleswig-Holstein'}, {'code': 'DE-TH', 'name': 'Thuringen'}]
[{'code': 'US-AL', 'name': 'Alabama'}, {'code': 'US-AK', 'name': 'Alaska'}, {'code': 'US-AZ', 'name': 'Arizona'}, {'code': 'US-AR', 'name': 'Arkansas'}, {'code': 'US-CA', 'name': 'California'}, {'code': 'US-CO', 'name': 'Colorado'}, {'code': 'US-CT', 'name': 'Connecticut'}, {'code': 'US-DE', 'name': 'Delaware'}, {'cod

In [12]:
countries['country'].unique()

array([None, 'US-VA', 'US-FL', 'CA-QC', 'US-NJ', 'CO-DC', 'US-NY',
       'DE-HE', 'IE-D', 'US-CA', 'DE-BE', 'DE-SN', 'PL-32', 'US-GA',
       'CN-BJ', 'RO-B', 'CO-SAN', 'ES-MD', 'US-IL', 'US-MO', 'CO-ANT',
       'US-TX', 'US-KS', 'US-UT', 'US-DE', 'US-IA', 'US-AZ', 'FR-HDF',
       'FR-IDF', 'US-WA', 'BG-22', 'CA-AB', 'NL-NH', 'US-MA'],
      dtype=object)

In [13]:
apache = sacar_pais(apache, countries)
duv = sacar_pais(duv, countries)
snmp=sacar_pais(snmp, countries)

In [14]:
apache.to_csv('Datos/Transformados/apache-logs-country.csv', index=False)
duv.to_csv('Datos/Transformados/Dataset-Unicauca-Version2-87Atts-country.csv', index=False)
snmp.to_csv('Datos/Transformados/reto8-registros_snmp-country.csv', index=False)